## Scrapping of outlet information for the 4 major supermarkets in Singapore

###### 1) Giant Hypermarket
###### 2) Cold Storage 
###### 3) Sheng Siong
###### 4) NTUC Fairprice

In [2]:
import requests
import re
from bs4 import BeautifulSoup
import json 
import numpy as np
import pandas as pd
from tqdm import tqdm

postalCodes = []

### 1) Scrapping of Giant Hypermarket Store Locator

In [2]:
supermarket_name_1 = "Giant Hypermart"
page = requests.get("https://giant.sg/store-locator")
soup = BeautifulSoup(page.content, 'html.parser')
scriptTags = soup.find_all('script', type="text/javascript")

scriptContent = []
for script in scriptTags:
    tagContent = script.get_text()
    if "function initMap()" in tagContent:
        scriptContent = tagContent        
        break;    

In [3]:
soup2 = BeautifulSoup(scriptContent, 'html.parser')
divTags = soup2.find_all('div', {"class": "map-info"})
for item in divTags:
    address = item.find('p', {'class': 'value'})
    postal_code = address.get_text().split(" ")[-1]

    if (len(postal_code) != 6):
        postal_code = postal_code.strip('[" ", "(",")","S"]')
    postalCodes.append({"supermarket" : supermarket_name_1,"postal_code" : postal_code})

### 2) Scrapping of Cold Storage Store Locator

In [4]:
supermarket_name_2 = "Cold Storage"
page = requests.get("https://coldstorage.com.sg/store-locator")
soup = BeautifulSoup(page.content, 'html.parser')
scriptTags = soup.find_all('script', type="text/javascript")
scriptContent = []
for script in scriptTags:
    tagContent = script.get_text()
    if "function initMap()" in tagContent:
        scriptContent = tagContent        
        break;    

In [5]:
soup2 = BeautifulSoup(scriptContent, 'html.parser')
divTags = soup2.find_all('div', {"class": "map-info"})
for item in divTags:
    address = item.find('p', {'class': 'value'})
    postal_code = address.get_text().split(" ")[-1]

    if (len(postal_code) != 6):
        postal_code = postal_code.strip('[" ", "(",")","S"]')
    postalCodes.append({"supermarket" : supermarket_name_2,"postal_code" : postal_code})

### 3) Scrapping of Sheng Siong Store Locator

In [6]:
supermarket_name_3 = "Sheng Siong"
page = requests.get("http://www.shengsiong.com.sg/pages/store-locator.html")
soup = BeautifulSoup(page.content, 'html.parser')
columns = soup.find_all('td')

In [7]:
for column in columns:
    outlet = column.find('div', {"class" : "map-location"})
    if (outlet):
        tags = outlet.find('a')     
        postal_code = tags.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.split(" ")[-1]
        postal_code = postal_code.strip('[" ", "(",")"]')
        if (len(postal_code) == 6):
            postalCodes.append({"supermarket" : supermarket_name_3,"postal_code" : postal_code})

### 4) Scrapping of NTUC Store Locator

In [8]:
supermarket_name_4 = "NTUC"
url = 'https://www.fairprice.com.sg/store-locator'
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
headers = {'User-Agent': user_agent}
page = requests.get(url,headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')

In [9]:
string = soup.find("script", {'id' : '__NEXT_DATA__'}).get_text()
res = json.loads(string) 
outlets = res["props"]["pageProps"]["data"]["fpstores"]

In [10]:
for outlet in outlets:
    postal_code = outlet['postalCode'] 
    postal_code = postal_code.strip('[" ", "(",")","S"]')
    if (len(postal_code) == 6):
        postalCodes.append({"supermarket" : outlet['storeType'],"postal_code" : postal_code})

### Using OneMap API to retrieve location details

In [11]:
def getPostalInfo(postal_code):
    request = requests.get(
            'https://developers.onemap.sg/commonapi/search?searchVal='+ str(postal_code) + '&returnGeom=Y&getAddrDetails=Y'
        )
    result = request.json()
    return result;

In [12]:
output_df = pd.DataFrame()
tq = tqdm(postalCodes)
for outlet in postalCodes:
    tq.set_description(f"Retrieving {outlet} information")
    results = getPostalInfo(outlet["postal_code"])['results']
    if (results):
        rowOutput = {'shop' : outlet['supermarket'], 'location' : results[0]['ROAD_NAME'].title(), 'latitude' : results[0]["LATITUDE"],'longitude':results[0]["LONGITUDE"]}
        output_df = output_df.append(rowOutput, ignore_index=True)
        
output_df.to_csv("retailer.csv")

Retrieving {'supermarket': 'FairPrice', 'postal_code': '310192'} information:   0%|          | 0/317 [07:00<?, ?it/s]        

In [4]:
socket.AF_INET

<AddressFamily.AF_INET: 2>

In [82]:
import socket

def try_port(port):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.settimeout(0.1)
    
    try:
        s.connect(('3.1.141.4', port))
    except socket.error:
        print ("Port is closed")
        return False

    try:
#         s.send(b"a\n")
        data = s.recv(2048)
        print ("Port is open! Received: ", data)
        s.close()
        return True
    except socket.timeout:
        print ("Port is open, but timed out!")
        s.close()
        return True

def scan():
    found_ports=[]
    for port in range(1,65535):
        print(port)
        connected = try_port(port)
        if connected:
            found_ports.append(port)
    return found_ports

# print (scan())

In [107]:
try_port(7111)
try_port(8888)
try_port(9888)

try_port(53572)
# try_port(52062)


Port is closed
Port is closed
Port is closed
Port is open! Received:  b'flag{C0M3_kN0cK_Kn0CK_KNOck_kn0CcK_0n_mY_p0rT}\n'


True

In [83]:
scan()

1
Port is closed
2
Port is closed
3
Port is closed
4
Port is closed
5
Port is closed
6
Port is closed
7
Port is closed
8
Port is closed
9
Port is closed
10
Port is closed
11
Port is closed
12
Port is closed
13
Port is closed
14
Port is closed
15
Port is closed
16
Port is closed
17
Port is closed
18
Port is closed
19
Port is closed
20
Port is closed
21
Port is closed
22
Port is open! Received:  b'SSH-2.0-OpenSSH_7.2p2 Ubuntu-4ubuntu2.8\r\n'
23
Port is closed
24
Port is closed
25
Port is closed
26
Port is closed
27
Port is closed
28
Port is closed
29
Port is closed
30
Port is closed
31
Port is closed
32
Port is closed
33
Port is closed
34
Port is closed
35
Port is closed
36
Port is closed
37
Port is closed
38
Port is closed
39
Port is closed
40
Port is closed
41
Port is closed
42
Port is closed
43
Port is closed
44
Port is closed
45
Port is closed
46
Port is closed
47
Port is closed
48
Port is closed
49
Port is closed
50
Port is closed
51
Port is closed
52
Port is closed
53
Port is close

Port is closed
436
Port is closed
437
Port is closed
438
Port is closed
439
Port is closed
440
Port is closed
441
Port is closed
442
Port is closed
443
Port is closed
444
Port is closed
445
Port is closed
446
Port is closed
447
Port is closed
448
Port is closed
449
Port is closed
450
Port is closed
451
Port is closed
452
Port is closed
453
Port is closed
454
Port is closed
455
Port is closed
456
Port is closed
457
Port is closed
458
Port is closed
459
Port is closed
460
Port is closed
461
Port is closed
462
Port is closed
463
Port is closed
464
Port is closed
465
Port is closed
466
Port is closed
467
Port is closed
468
Port is closed
469
Port is closed
470
Port is closed
471
Port is closed
472
Port is closed
473
Port is closed
474
Port is closed
475
Port is closed
476
Port is closed
477
Port is closed
478
Port is closed
479
Port is closed
480
Port is closed
481
Port is closed
482
Port is closed
483
Port is closed
484
Port is closed
485
Port is closed
486
Port is closed
487
Port is clos

Port is closed
868
Port is closed
869
Port is closed
870
Port is closed
871
Port is closed
872
Port is closed
873
Port is closed
874
Port is closed
875
Port is closed
876
Port is closed
877
Port is closed
878
Port is closed
879
Port is closed
880
Port is closed
881
Port is closed
882
Port is closed
883
Port is closed
884
Port is closed
885
Port is closed
886
Port is closed
887
Port is closed
888
Port is closed
889
Port is closed
890
Port is closed
891
Port is closed
892
Port is closed
893
Port is closed
894
Port is closed
895
Port is closed
896
Port is closed
897
Port is closed
898
Port is closed
899
Port is closed
900
Port is closed
901
Port is closed
902
Port is closed
903
Port is closed
904
Port is closed
905
Port is closed
906
Port is closed
907
Port is closed
908
Port is closed
909
Port is closed
910
Port is closed
911
Port is closed
912
Port is closed
913
Port is closed
914
Port is closed
915
Port is closed
916
Port is closed
917
Port is closed
918
Port is closed
919
Port is clos

Port is closed
1285
Port is closed
1286
Port is closed
1287
Port is closed
1288
Port is closed
1289
Port is closed
1290
Port is closed
1291
Port is closed
1292
Port is closed
1293
Port is closed
1294
Port is closed
1295
Port is closed
1296
Port is closed
1297
Port is closed
1298
Port is closed
1299
Port is closed
1300
Port is closed
1301
Port is closed
1302
Port is closed
1303
Port is closed
1304
Port is closed
1305
Port is closed
1306
Port is closed
1307
Port is closed
1308
Port is closed
1309
Port is closed
1310
Port is closed
1311
Port is closed
1312
Port is closed
1313
Port is closed
1314
Port is closed
1315
Port is closed
1316
Port is closed
1317
Port is closed
1318
Port is closed
1319
Port is closed
1320
Port is closed
1321
Port is closed
1322
Port is closed
1323
Port is closed
1324
Port is closed
1325
Port is closed
1326
Port is closed
1327
Port is closed
1328
Port is closed
1329
Port is closed
1330
Port is closed
1331
Port is closed
1332
Port is closed
1333
Port is closed
1334


Port is closed
1695
Port is closed
1696
Port is closed
1697
Port is closed
1698
Port is closed
1699
Port is closed
1700
Port is closed
1701
Port is closed
1702
Port is closed
1703
Port is closed
1704
Port is closed
1705
Port is closed
1706
Port is closed
1707
Port is closed
1708
Port is closed
1709
Port is closed
1710
Port is closed
1711
Port is closed
1712
Port is closed
1713
Port is closed
1714
Port is closed
1715
Port is closed
1716
Port is closed
1717
Port is closed
1718
Port is closed
1719
Port is closed
1720
Port is closed
1721
Port is closed
1722
Port is closed
1723
Port is closed
1724
Port is closed
1725
Port is closed
1726
Port is closed
1727
Port is closed
1728
Port is closed
1729
Port is closed
1730
Port is closed
1731
Port is closed
1732
Port is closed
1733
Port is closed
1734
Port is closed
1735
Port is closed
1736
Port is closed
1737
Port is closed
1738
Port is closed
1739
Port is closed
1740
Port is closed
1741
Port is closed
1742
Port is closed
1743
Port is closed
1744


...

Port is closed
64216
Port is closed
64217
Port is closed
64218
Port is closed
64219
Port is closed
64220
Port is closed
64221
Port is closed
64222
Port is closed
64223
Port is closed
64224
Port is closed
64225
Port is closed
64226
Port is closed
64227
Port is closed
64228
Port is closed
64229
Port is closed
64230
Port is closed
64231
Port is closed
64232
Port is closed
64233
Port is closed
64234
Port is closed
64235
Port is closed
64236
Port is closed
64237
Port is closed
64238
Port is closed
64239
Port is closed
64240
Port is closed
64241
Port is closed
64242
Port is closed
64243
Port is closed
64244
Port is closed
64245
Port is closed
64246
Port is closed
64247
Port is closed
64248
Port is closed
64249
Port is closed
64250
Port is closed
64251
Port is closed
64252
Port is closed
64253
Port is closed
64254
Port is closed
64255
Port is closed
64256
Port is closed
64257
Port is closed
64258
Port is closed
64259
Port is closed
64260
Port is closed
64261
Port is closed
64262
Port is close

64606
Port is closed
64607
Port is closed
64608
Port is closed
64609
Port is closed
64610
Port is closed
64611
Port is closed
64612
Port is closed
64613
Port is closed
64614
Port is closed
64615
Port is closed
64616
Port is closed
64617
Port is closed
64618
Port is closed
64619
Port is closed
64620
Port is closed
64621
Port is closed
64622
Port is closed
64623
Port is closed
64624
Port is closed
64625
Port is closed
64626
Port is closed
64627
Port is closed
64628
Port is closed
64629
Port is closed
64630
Port is closed
64631
Port is closed
64632
Port is closed
64633
Port is closed
64634
Port is closed
64635
Port is closed
64636
Port is closed
64637
Port is closed
64638
Port is closed
64639
Port is closed
64640
Port is closed
64641
Port is closed
64642
Port is closed
64643
Port is closed
64644
Port is closed
64645
Port is closed
64646
Port is closed
64647
Port is closed
64648
Port is closed
64649
Port is closed
64650
Port is closed
64651
Port is closed
64652
Port is closed
64653
Port is

Port is closed
64997
Port is closed
64998
Port is closed
64999
Port is closed
65000
Port is closed
65001
Port is closed
65002
Port is closed
65003
Port is closed
65004
Port is closed
65005
Port is closed
65006
Port is closed
65007
Port is closed
65008
Port is closed
65009
Port is closed
65010
Port is closed
65011
Port is closed
65012
Port is closed
65013
Port is closed
65014
Port is closed
65015
Port is closed
65016
Port is closed
65017
Port is closed
65018
Port is closed
65019
Port is closed
65020
Port is closed
65021
Port is closed
65022
Port is closed
65023
Port is closed
65024
Port is closed
65025
Port is closed
65026
Port is closed
65027
Port is closed
65028
Port is closed
65029
Port is closed
65030
Port is closed
65031
Port is closed
65032
Port is closed
65033
Port is closed
65034
Port is closed
65035
Port is closed
65036
Port is closed
65037
Port is closed
65038
Port is closed
65039
Port is closed
65040
Port is closed
65041
Port is closed
65042
Port is closed
65043
Port is close

Port is closed
65388
Port is closed
65389
Port is closed
65390
Port is closed
65391
Port is closed
65392
Port is closed
65393
Port is closed
65394
Port is closed
65395
Port is closed
65396
Port is closed
65397
Port is closed
65398
Port is closed
65399
Port is closed
65400
Port is closed
65401
Port is closed
65402
Port is closed
65403
Port is closed
65404
Port is closed
65405
Port is closed
65406
Port is closed
65407
Port is closed
65408
Port is closed
65409
Port is closed
65410
Port is closed
65411
Port is closed
65412
Port is closed
65413
Port is closed
65414
Port is closed
65415
Port is closed
65416
Port is closed
65417
Port is closed
65418
Port is closed
65419
Port is closed
65420
Port is closed
65421
Port is closed
65422
Port is closed
65423
Port is closed
65424
Port is closed
65425
Port is closed
65426
Port is closed
65427
Port is closed
65428
Port is closed
65429
Port is closed
65430
Port is closed
65431
Port is closed
65432
Port is closed
65433
Port is closed
65434
Port is close

[22, 80, 12561, 15151, 18777, 24242, 24680, 55555]